# HomeMatch application from **"Future Homes Realty"**

**Step 1: Setting Up the Python Application**


In [1]:
# Install libraries
! pip install -U --quiet openai
! pip install -U --quiet lancedb pydantic
! pip install "ipywidgets>=7,<8"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.5/25.5 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.9 MB/s eta 0:00:00


In [2]:
#Import statments
from google.colab import userdata, output
from bs4 import BeautifulSoup
from markdown import markdown
from lancedb.pydantic import Vector, LanceModel
from lancedb.embeddings import get_registry
from ipywidgets import Layout, Button, Box, Textarea, Label, IntSlider, FloatSlider
import openai
import random
import json
import pandas as pd
import os
import lancedb


In [3]:
# Setting up keys and constants

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
DATA_FILE = "listings.json"


**Step 2: Generating Real Estate Listings**



In [4]:
# Define user and system mensages

system_prompt = """
You are a expert real estate agent in Lisbon, Portugal.
You are focused on selling houses in the following neighborhoods: Rato, São Bento, Ajuda, Lapa, Estrela, Chiado, Baixa, Príncipe Real, Campo de Ourique.
"""

human_prompt = """
Generate at least 25 real estate listings in the neighborhoods that you are working.
Use your imagination to generate the listings, be creative and but make sure the listings seem real.
The listings must be a JSON array of dictionaries with each item in the format as shown below:

        {

          "neighborhood": "Chiado",
          "price": 800000,
          "bedrooms": 3,
          "bathrooms": 2,
          "house_size": 250,
          "description": "Welcome to this eco-friendly oasis nestled in the heart of Chiado. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such\
as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept\
kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising\
on style in this Chiado gem. Chiado is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths.\
Take a stroll through the nearby Chiado park or grab a cup of coffee at at the cozy corner Cafe. With easy access to public transportation and bike lanes, commuting is a breeze."

        }
"""

In [5]:
# check the prompts
print(system_prompt)
print(human_prompt)


You are a expert real estate agent in Lisbon, Portugal.
You are focused on selling houses in the following neighborhoods: Rato, São Bento, Ajuda, Lapa, Estrela, Chiado, Baixa, Príncipe Real, Campo de Ourique.


Generate at least 25 real estate listings in the neighborhoods that you are working.
Use your imagination to generate the listings, be creative and but make sure the listings seem real. 
The listings must be a JSON array of dictionaries with each item in the format as shown below:

        {

          "neighborhood": "Chiado",
          "price": 800000,
          "bedrooms": 3,
          "bathrooms": 2,
          "house_size": 250,
          "description": "Welcome to this eco-friendly oasis nestled in the heart of Chiado. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features suchas solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-conceptkitch

In [6]:
# Creates de listings and saves them in json format to the datafile

# if the file already existes loads the data from it
if os.path.exists(DATA_FILE):
  f= open(DATA_FILE, 'r')
  real_estate_listings = f.read()
  print(real_estate_listings)
  f.close()
# if not calls the model to generate the listings and saves the content to the file in json format
else:
  completion = openai.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
          {
            "role": "system",
            "content": system_prompt
          },
          {
            "role": "user",
            "content": human_prompt
          }
          ],
  # temperatue and top_p adusted acording open AI best practices for this scenario
  temperature=0.7,
  top_p=0.8,
  )

  real_estate_listings=completion.choices[0].message.content
  md_text = markdown(real_estate_listings)
  real_estate_listings_text = ''.join(BeautifulSoup(md_text).findAll(string=True))
  real_estate_listings_json = real_estate_listings_text.replace('json\n', '')
  #print(md_text)
  #print(real_estate_listings_text)
  print(real_estate_listings_json)
  f= open(DATA_FILE, 'w')
  f.write(real_estate_listings_json)
  f.close()

[
  {
    "neighborhood": "Rato",
    "price": 950000,
    "bedrooms": 4,
    "bathrooms": 3,
    "house_size": 300,
    "description": "Luxurious 4-bedroom, 3-bathroom townhouse in the heart of Rato. This beautifully renovated property features high-end finishes, a modern kitchen, and a rooftop terrace with stunning city views. Enjoy the convenience of living in a vibrant neighborhood with easy access to shops, restaurants, and public transportation."
  },
  {
    "neighborhood": "São Bento",
    "price": 1200000,
    "bedrooms": 5,
    "bathrooms": 4,
    "house_size": 350,
    "description": "Elegant 5-bedroom, 4-bathroom villa in São Bento. This spacious home offers a blend of traditional architecture and modern amenities. Relax in the private garden or entertain guests in the stylish living areas. Enjoy the tranquility of São Bento while being just a short walk away from cultural attractions and gourmet restaurants."
  },
  {
    "neighborhood": "Ajuda",
    "price": 650000,
    "

In [7]:
# create a pandas data frame from the json file to help in the visualization

data_frame = pd.read_json(DATA_FILE)
#data_frame.head()
data_frame

,neighborhood,price,bedrooms,bathrooms,house_size,description
0,Rato,950000,4,3,300,"Luxurious 4-bedroom, 3-bathroom townhouse in t..."
1,São Bento,1200000,5,4,350,"Elegant 5-bedroom, 4-bathroom villa in São Ben..."
2,Ajuda,650000,3,2,200,"Charming 3-bedroom, 2-bathroom house with pano..."
3,Lapa,1800000,6,5,400,"Stunning 6-bedroom, 5-bathroom mansion in Lapa..."
4,Estrela,850000,3,3,220,"Modern 3-bedroom, 3-bathroom duplex in Estrela..."
5,Chiado,800000,3,2,250,Welcome to this eco-friendly oasis nestled in ...
6,Baixa,700000,2,1,180,"Cozy 2-bedroom, 1-bathroom apartment in the hi..."
7,Príncipe Real,1300000,4,3,280,"Spacious 4-bedroom, 3-bathroom townhouse in Pr..."
8,Campo de Ourique,950000,3,2,210,"Bright and airy 3-bedroom, 2-bathroom penthous..."
9,Rato,1100000,4,3,320,"Contemporary 4-bedroom, 3-bathroom villa in Ra..."


**Step 3: Storing Listings in a Vector Database**



In [8]:
#Create the database, the schema and the table

db = lancedb.connect("~/lancedb")

#drops the table if already exists so that we can run this code any time
if "listings" in db:
  db.drop_table("listings")

embeddings = get_registry().get("openai").create()

class RealEstateListing(LanceModel):
    neighborhood: str
    price: int
    bedrooms: int
    bathrooms: int
    house_size: int
    description: str = embeddings.SourceField()
    description_vector: Vector(embeddings.ndims()) = embeddings.VectorField()

if "listings" not in db:
  table = db.create_table("listings",schema=RealEstateListing)

table.list_versions()

[{'version': 1,
  'timestamp': datetime.datetime(2024, 7, 12, 9, 57, 40, 820959),
  'metadata': {}}]

In [9]:
# add data to the table

# only adds data if the table is empty
if len(table) == 0:
  with open(DATA_FILE) as f:
    data = json.load(f)

  table.add(data)

len(table)


27

In [10]:
#check the data in the table header
table.head().to_pandas()

,neighborhood,price,bedrooms,bathrooms,house_size,description,description_vector
0,Rato,950000,4,3,300,"Luxurious 4-bedroom, 3-bathroom townhouse in t...","[-0.0026460392, 0.036372222, 0.00012096477, -0..."
1,São Bento,1200000,5,4,350,"Elegant 5-bedroom, 4-bathroom villa in São Ben...","[-0.007843574, 0.03005826, 0.008679256, -0.002..."
2,Ajuda,650000,3,2,200,"Charming 3-bedroom, 2-bathroom house with pano...","[-0.002422086, 0.03324729, -0.0049791895, -0.0..."
3,Lapa,1800000,6,5,400,"Stunning 6-bedroom, 5-bathroom mansion in Lapa...","[0.006552568, 0.034542102, 0.0024477225, -0.03..."
4,Estrela,850000,3,3,220,"Modern 3-bedroom, 3-bathroom duplex in Estrela...","[-0.0029958547, 0.028802484, 0.00191411, -0.01..."


**Step 4: Building the User Preference Interface**

In [11]:
max_price = 5000000 # changhe this value if you want a diferent maximum house price
max_bedrooms = 10 #changhe this value if you want a diferent maximum number of bedrooms
max_bathrooms = 10 #changhe this value if you want a diferent maximum number of bathrooms
max_square_feet = 1000 # #changhe this value if you want a diferent house size maximum
results_list = []

#define the layout for the form
form_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
    )

# define relevant input fields for the form
form_fields = [
    Box([Label(value='Maximum Price'), IntSlider(min=100000, max=max_price, step=50000)], layout=form_layout),
    Box([Label(value='Bed Rooms minimum'), IntSlider(min=1, max=max_bedrooms, step=1)], layout=form_layout),
    Box([Label(value='Bath Rooms minimum'), IntSlider(min=1, max= max_bathrooms, step=1)], layout=form_layout),
    Box([Label(value='House Size minimum (m2)'), IntSlider(min=50, max=max_square_feet, step=50)], layout=form_layout),
    Box([Label(value='Description'),Textarea(placeholder="Describe the type of house you are looking for")], layout=form_layout),
    Button(description='Search', disabled=False, button_style='', tooltip='Search')
  ]

# define the form
form = Box(form_fields, layout=Layout(
      display='flex',
      flex_flow='column',
      align_items='stretch',
      width='50%'
  ))

# function to return the user inputs in the "results_list"
def return_results(_):
  results_list.clear()
  results_list.append(form_fields[0].children[1].value)
  results_list.append(form_fields[1].children[1].value)
  results_list.append(form_fields[2].children[1].value)
  results_list.append(form_fields[3].children[1].value)
  results_list.append(form_fields[4].children[1].value)
  return results_list


form.children[5].on_click(return_results)

form

# CLICK SEARCH WHEN YOU ARE READY TO ADVANCE


Box(children=(Box(children=(Label(value='Maximum Price'), IntSlider(value=100000, max=5000000, min=100000, ste…

In [21]:
max_price = results_list[0]
min_bedrooms = results_list[1]
min_bathrooms = results_list[2]
min_house_size = results_list[3]
description = results_list[4]

# test if the inputs are ok
print("price:", max_price)
print("bedrooms:",min_bedrooms)
print("bathrooms:", min_bathrooms)
print("House Size:", min_house_size)
print("Description:", description)


price: 2000000
bedrooms: 4
bathrooms: 3
House Size: 300
Description: A spacious modern house.



Support for third party widgets will remain active for the duration of the session. To disable support:

**Step 5: Searching Based on Preferences**



In [22]:
# Get the 5 more relevant listings from the database acording to the semantic closeness to the buyer’s preferences.

# open the datbase and table
db = lancedb.connect("~/lancedb")
table = db.open_table("listings")
#print(table.head().to_pandas())

# define the query with the SQL filters
query = f"(price <= {max_price}) AND (bedrooms >= {min_bedrooms}) AND (bathrooms >= {min_bathrooms}) AND (house_size >= {min_house_size})"
#print(query)

# search the database acording the preferences (vector search) and SQL filters
results = table.search(description, vector_column_name="description_vector").where(query, prefilter=True).limit(5).to_pandas()

# show the results
results



,neighborhood,price,bedrooms,bathrooms,house_size,description,description_vector,_distance
0,Rato,1100000,4,3,320,"Contemporary 4-bedroom, 3-bathroom villa in Ra...","[-0.008393557, 0.034873877, -0.0011710366, -0....",0.302403
1,Príncipe Real,1350000,5,4,320,"Stylish 5-bedroom, 4-bathroom townhouse in Prí...","[-0.00034730663, 0.02269015, 0.008649216, -0.0...",0.307160
2,Lapa,1800000,6,5,400,"Stunning 6-bedroom, 5-bathroom mansion in Lapa...","[0.006552568, 0.034542102, 0.0024477225, -0.03...",0.307924
3,Rato,950000,4,3,300,"Luxurious 4-bedroom, 3-bathroom townhouse in t...","[-0.0026460392, 0.036372222, 0.00012096477, -0...",0.311840
4,São Bento,1200000,5,4,350,"Elegant 5-bedroom, 4-bathroom villa in São Ben...","[-0.007843574, 0.03005826, 0.008679256, -0.002...",0.313885


**Step 6: Personalizing Listing Descriptions**


In [23]:
# create a string with the filtered list to be the context for the final prompt

listings_string = ""

for index, row in results.iterrows():
      listings_string += "Located in " + row["neighborhood"] + ", with " + str(row["house_size"]) + " m2," + " and a price of " + str(row["price"]) + " EUR. " + row["description"] + "\n"


print(listings_string)

Located in Rato, with 320 m2, and a price of 1100000 EUR. Contemporary 4-bedroom, 3-bathroom villa in Rato. This sleek and stylish home boasts a private swimming pool, a rooftop deck, and a state-of-the-art home automation system. Live in luxury in the heart of Rato with easy access to upscale shopping, gourmet dining, and vibrant nightlife.
Located in Príncipe Real, with 320 m2, and a price of 1350000 EUR. Stylish 5-bedroom, 4-bathroom townhouse in Príncipe Real. This contemporary home boasts a rooftop terrace, a gourmet kitchen, and a private garage. Enjoy the cosmopolitan vibe of Príncipe Real with its fashionable boutiques, upscale eateries, and cultural hotspots just a short walk away.
Located in Lapa, with 400 m2, and a price of 1800000 EUR. Stunning 6-bedroom, 5-bathroom mansion in Lapa. This grand property boasts a private swimming pool, a wine cellar, and a lush garden perfect for outdoor entertaining. Enjoy the luxury of Lapa living with easy access to upscale boutiques, art 

In [24]:
# Final query with LLM Augementation

query = """
    You are a expert real estate agent in Lisbon, Portugal.
    Based on the listings in the context, tell me listing that is the best fit for my customer question, emphasizing aspects of the listing that align with what the customer is looking for.
    Make sure you do not paraphrase the context but maintain factual integrity, ensuring that the answer is appealing without altering factual information.
    """

context = listings_string

question = description

response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content":query

            },
            {"role": "user", "content": f"CONTEXT: {context}\nQUESTION: {question}"},
            {"role": "user", "content": "ANSWER:"},
        ],
    )

response = response.choices[0].message.content
print(response)

Based on your criteria of looking for a spacious modern house, I recommend the "Luxurious 4-bedroom, 3-bathroom townhouse in the heart of Rato" listed at 950000 EUR. This property offers 300 m2 of space, high-end finishes, a modern kitchen, and a rooftop terrace with stunning city views. It perfectly combines modern amenities with a vibrant neighborhood lifestyle, providing easy access to shops, restaurants, and public transportation.


**Step 7: Deliverables and Testing**


# Test Nº 1

**Customer input**

price: 900000
bedrooms: 3
bathrooms: 2
House Size: 200
Description: A house in a neighborhood with gardens and good access to public transportation


**Filtered results**

Located in Chiado, with 250 m2, and a price of 800000 EUR. Welcome to this eco-friendly oasis nestled in the heart of Chiado. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Chiado gem. Chiado is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Chiado park or grab a cup of coffee at the cozy corner Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.

Located in Ajuda, with 220 m2, and a price of 750000 EUR. Spacious 4-bedroom, 3-bathroom house with garden in Ajuda. This family-friendly home offers a bright and airy living space, a modern kitchen, and a covered patio for outdoor dining. Enjoy the peaceful surroundings of Ajuda while still being close to schools, parks, and amenities.

Located in Ajuda, with 190 m2, and a price of 720000 EUR. Quaint 3-bedroom, 2-bathroom cottage in Ajuda. This charming home offers a peaceful retreat with a garden, a cozy fireplace, and a covered patio for outdoor dining. Enjoy the tranquility of Ajuda while still being just a short drive away from the city center and the waterfront.

Located in Baixa, with 180 m2, and a price of 680000 EUR. Renovated 3-bedroom, 2-bathroom apartment in Baixa. This modern property boasts an open-plan layout, designer finishes, and views of the historic streets below. Experience the vibrant energy of Baixa with its bustling markets, street performers, and cultural events right at your doorstep.

Located in Estrela, with 240 m2, and a price of 900000 EUR. Spacious 4-bedroom, 3-bathroom apartment in Estrela. This contemporary home offers panoramic city views, a modern kitchen, and a private balcony for outdoor relaxation. Immerse yourself in the vibrant culture of Estrela with its historic architecture, trendy cafes, and lush green parks nearby.


**Final LLM Recommendation**

The listing in Chiado for 800,000 EUR is the perfect fit for you! This eco-friendly 3-bedroom, 2-bathroom home features a spacious backyard with a vegetable garden, ideal for those who appreciate green spaces. It is located in a close-knit, environmentally-conscious community with easy access to public transportation. Enjoy the blend of sustainable living and style in this charming Chiado gem, where natural light highlights the beautiful hardwood floors and eco-conscious finishes.

# Test Nº2

**Customer Input**

price: 800000
bedrooms: 2
bathrooms: 2
House Size: 150
Description: A house in Ajuda

**Filtered Listings**

Located in Ajuda, with 200 m2, and a price of 650000 EUR. Charming 3-bedroom, 2-bathroom house with panoramic views in Ajuda. This bright and airy property features a cozy fireplace, a modern kitchen, and a spacious terrace overlooking the river. Experience the peaceful surroundings of Ajuda while still being close to schools, parks, and historical landmarks.

Located in Ajuda, with 220 m2, and a price of 750000 EUR. Spacious 4-bedroom, 3-bathroom house with garden in Ajuda. This family-friendly home offers a bright and airy living space, a modern kitchen, and a covered patio for outdoor dining. Enjoy the peaceful surroundings of Ajuda while still being close to schools, parks, and amenities.

Located in Ajuda, with 190 m2, and a price of 720000 EUR. Quaint 3-bedroom, 2-bathroom cottage in Ajuda. This charming home offers a peaceful retreat with a garden, a cozy fireplace, and a covered patio for outdoor dining. Enjoy the tranquility of Ajuda while still being just a short drive away from the city center and the waterfront.

Located in Baixa, with 190 m2, and a price of 700000 EUR. Charming 2-bedroom, 2-bathroom flat in Baixa. This cozy property offers a modern kitchen, a spacious living area, and views of the historic streets below. Experience the lively atmosphere of Baixa with its vibrant markets, cultural events, and traditional Fado music venues nearby.

Located in Baixa, with 180 m2, and a price of 680000 EUR. Renovated 3-bedroom, 2-bathroom apartment in Baixa. This modern property boasts an open-plan layout, designer finishes, and views of the historic streets below. Experience the vibrant energy of Baixa with its bustling markets, street performers, and cultural events right at your doorstep.



**Final LLM Recomendation**

I would recommend the spacious 4-bedroom, 3-bathroom house in Ajuda, priced at 750000 EUR. This family-friendly home features a bright and airy living space, a modern kitchen, and a covered patio for outdoor dining. It also includes a garden, perfect for relaxing or entertaining. Enjoy the peaceful surroundings of Ajuda while still being conveniently close to schools, parks, and amenities.

# Test Nº3

**Customer Input**

price: 2000000
bedrooms: 4
bathrooms: 3
House Size: 300
Description: A spacious modern house.

**Filtered Listings**

Located in Rato, with 320 m2, and a price of 1100000 EUR. Contemporary 4-bedroom, 3-bathroom villa in Rato. This sleek and stylish home boasts a private swimming pool, a rooftop deck, and a state-of-the-art home automation system. Live in luxury in the heart of Rato with easy access to upscale shopping, gourmet dining, and vibrant nightlife.

Located in Príncipe Real, with 320 m2, and a price of 1350000 EUR. Stylish 5-bedroom, 4-bathroom townhouse in Príncipe Real. This contemporary home boasts a rooftop terrace, a gourmet kitchen, and a private garage. Enjoy the cosmopolitan vibe of Príncipe Real with its fashionable boutiques, upscale eateries, and cultural hotspots just a short walk away.

Located in Lapa, with 400 m2, and a price of 1800000 EUR. Stunning 6-bedroom, 5-bathroom mansion in Lapa. This grand property boasts a private swimming pool, a wine cellar, and a lush garden perfect for outdoor entertaining. Enjoy the luxury of Lapa living with easy access to upscale boutiques, art galleries, and gourmet dining options.

Located in Rato, with 300 m2, and a price of 950000 EUR. Luxurious 4-bedroom, 3-bathroom townhouse in the heart of Rato. This beautifully renovated property features high-end finishes, a modern kitchen, and a rooftop terrace with stunning city views. Enjoy the convenience of living in a vibrant neighborhood with easy access to shops, restaurants, and public transportation.

Located in São Bento, with 350 m2, and a price of 1200000 EUR. Elegant 5-bedroom, 4-bathroom villa in São Bento. This spacious home offers a blend of traditional architecture and modern amenities. Relax in the private garden or entertain guests in the stylish living areas. Enjoy the tranquility of São Bento while being just a short walk away from cultural attractions and gourmet restaurants.



**Final LLM Recomendation**

Based on your criteria of looking for a spacious modern house, I recommend the "Luxurious 4-bedroom, 3-bathroom townhouse in the heart of Rato" listed at 950000 EUR. This property offers 300 m2 of space, high-end finishes, a modern kitchen, and a rooftop terrace with stunning city views. It perfectly combines modern amenities with a vibrant neighborhood lifestyle, providing easy access to shops, restaurants, and public transportation.

**Step 8: Project Submission**

**Generated Listings**: Include a file that contains your synthetically generated real estate listings. Name this file "listings" - Please check the file in the github folder I have sent.

**Project Documentation**: Include a readme file or an accompanying document explaining the functionality, how to run the code, and any prerequisites or dependencies.

**Instructions**

If you are running the program ***for the first time***, go through all the steps below running every cell in the notebook.

Step 1: Setting Up the Python Application

Step 2: Generating Real Estate Listings

Step 3: Storing Listings in a Vector Database

Step 4: Building the User Preference Interface - In this step please input your preferences

Step 5: Searching Based on Preferences

Step 6: Personalizing Listing Descriptions


***If you are happy with the synthetic data generated***  by the LLM and just want to look for a different listing do this:

1.   Go to user interface in step 4
2.   Change your inputs (preferences) and click search
3.   Run step 5 and 6 again to obtain the final recommendation

For further information about the functionalities please refer to the read.me file